In [1]:
from search_space import distribution_names as df
from search_space import NaturalSearchSpace as N
from search_space import ContinueSearchSpace as R
from search_space import UniversalVariable as x

In [2]:
n1 = N(1,15)
n = N(10, 100, distribute_like=df.UNIFORM) | (x >= 5*n1)
m = N(10, 100) | (n-2 < x, x > n)

In [3]:
value, context = n.get_sampler()
value

72

In [4]:
value, _ = m.get_sampler()
value

95

In [5]:
nv, context = n.get_sampler()
mv, _ = m.get_sampler(context=context)
n1v, _ = n1.get_sampler(context=context)
print(mv, '<', nv, '>', n1v)

72 < 59 > 3


In [6]:
k = m + n
kv, context = k.get_sampler()
nv, _ = n.get_sampler(context=context)
mv, _ = m.get_sampler(context=context)

print(kv, '=', mv, '+', nv, '|', mv, '<', nv)

95 = 63 + 32 | 63 < 32


In [7]:
k = m + n + 2 + n + 4 + m
kv, context = k.get_sampler()
print(kv)

274


In [8]:
context.print_history()

Sampler in NaturalSearchSpace[(10, 100)]:
	Sampler in NaturalSearchSpace[(10, 100)]:
		Sampler in NaturalSearchSpace[(1, 15)]:
			NaturalSearchSpace[(1, 15)] ---> 11.485438890681262 ---> 11
		Transform[GreatEqual]: (10, 100) ---> (55, 100)
		NaturalSearchSpace[(55, 100)] ---> 60.67454180366862 ---> 61
	Transform[Great]: (10, 100) ---> (59, 100)
	Transform[Great]: (59, 100) ---> (61, 100)
	NaturalSearchSpace[(61, 100)] ---> 73.00573745111024 ---> 73


In [10]:
from search_space import Function

class Factorial(Function):
    def __call__(self, n: float):
        result = 1
        for i in range(2, int(n+1)):
            result *= i
        return result


k_factorial = Factorial(n1) - Factorial(n1-1)
kfv, context = k_factorial.get_sampler()
kv, _ = k.get_sampler(context=context)
print(kv)
print(kfv)
context.print_history()

342
96
Sampler in NaturalSearchSpace[(1, 15)]:
	NaturalSearchSpace[(1, 15)] ---> 4.712011682471966 ---> 5
Sampler in NaturalSearchSpace[(10, 100)]:
	Sampler in NaturalSearchSpace[(10, 100)]:
		Transform[GreatEqual]: (10, 100) ---> (25, 100)
		NaturalSearchSpace[(25, 100)] ---> 78.13744729812177 ---> 78
	Transform[Great]: (10, 100) ---> (76, 100)
	Transform[Great]: (76, 100) ---> (78, 100)
	NaturalSearchSpace[(78, 100)] ---> 90.31162383613366 ---> 90


In [17]:
validate_replay_count = 100
"""Real Space Test"""
n = R() | (x < 5)
for _ in range(validate_replay_count):
    value, _ = n.get_sampler()
    assert value < 5
"""Natural Space Test"""
n = N() | (x < 5)
for _ in range(validate_replay_count):
    value, _ = n.get_sampler()
    assert value < 5
"""Context Sensitive Test"""
n = N() | (1 < x, x < 10)
m = R() | (x < n)
for _ in range(validate_replay_count):
    nv, context = n.get_sampler()
    mv, _ = m.get_sampler(context)
    assert nv < 10
    assert mv < nv


KeyboardInterrupt: 